<a href="https://colab.research.google.com/github/masies/CRA/blob/main/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
!pip3 install --upgrade pip
#!pip install -qU t5
!pip3 install git+https://github.com/google-research/text-to-text-transfer-transformer.git #extra_id_x support

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://example_comment" 

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 1.6MB 7.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
  Cloning https://github.com/google-research/text-to-text-transfer-transformer.git to /tmp/pip-req-build-30e_p2iv
  Running command git clone -q https://github.com/google-research/text-to-text-transfer-transformer.git /tmp/pip-req-build-30e_p2iv
     |████████████████████████████████| 366 kB 6.5 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 1.2 MB 21.0 MB/s 
     |███

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
nq_tsv_path_code_code = {
    "train":      'gs://code_review_automation/dataset/new/fineTuningDatasets/code_code/train.tsv',
    "validation": 'gs://code_review_automation/dataset/new/fineTuningDatasets/code_code/val.tsv'
}
num_nq_examples_code_code = dict(train=134442, validation=16805)

nq_tsv_path_code_comment = {
    "train":      'gs://code_review_automation/dataset/new/fineTuningDatasets/code_comment/train.tsv',
    "validation": 'gs://code_review_automation/dataset/new/fineTuningDatasets/code_comment/val.tsv'
}
num_nq_examples_code_comment = dict(train=134442, validation=16805)

nq_tsv_path_codeANDcomment_code = {
    "train":      'gs://code_review_automation/dataset/new/fineTuningDatasets/codeANDcomment_code/train.tsv',
    "validation": 'gs://code_review_automation/dataset/new/fineTuningDatasets/codeANDcomment_code/val.tsv'
}
num_nq_examples_codeANDcomment_code = dict(train=134442, validation=16805)

nq_tsv_path_marked_code = {
    "train":      'gs://code_review_automation/dataset/new/fineTuningDatasets/marked_code/train.tsv',
    "validation": 'gs://code_review_automation/dataset/new/fineTuningDatasets/marked_code/val.tsv'
}
num_nq_examples_marked_code = dict(train=134442, validation=16805)



In [3]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary

# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase

vocab_model_path = 'gs://code_review_automation/CodeReviewModel/TestModel.model'
vocab_path = 'gs://code_review_automation/CodeReviewModel/TestModel.vocab'

TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask

def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}


############### FIRST TASK ###############
print("1st TASK : code 2 code")

def nq_dataset_code_code(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_code_code[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw validation examples...")
for ex in tfds.as_numpy(nq_dataset_code_code("validation").take(3)):
  print(ex)
print("A few raw training examples...")
for ex in tfds.as_numpy(nq_dataset_code_code("train").take(3)):
  print(ex)

def code_code_preprocessing(ds):
  def to_inputs_and_targets(ex):
        inputs = tf.strings.join(['code2code: ' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
t5.data.TaskRegistry.remove('code_code')
t5.data.TaskRegistry.add(
    "code_code",
    dataset_fn=nq_dataset_code_code,
    splits=["train", "validation"],
    text_preprocessor=[code_code_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_code_code
)

nq_task = t5.data.TaskRegistry.get("code_code")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 512, "targets": 512})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(3)):
  print(ex)




############### SECOND TASK ###############
print("2nd TASK : code 2 comment")

def nq_dataset_code_comment(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_code_comment[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw validation examples...")
for ex in tfds.as_numpy(nq_dataset_code_comment("validation").take(3)):
  print(ex)
print("A few raw training examples...")
for ex in tfds.as_numpy(nq_dataset_code_comment("train").take(3)):
  print(ex)

def code_comment_preprocessing(ds):
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['code2comment: ' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

#Create a new training task
t5.data.TaskRegistry.remove('code_comment')
t5.data.TaskRegistry.add(
    "code_comment",
    dataset_fn=nq_dataset_code_comment,
    splits=["train", "validation"],
    text_preprocessor=[code_comment_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_code_comment
)

nq_task = t5.data.TaskRegistry.get("code_comment")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 512, "targets": 512})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(3)):
  print(ex)



############### THIRD TASK ###############
print("3rd TASK : code and comment 2 code")

def nq_dataset_codeANDcomment_code(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_codeANDcomment_code[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw validation examples...")
for ex in tfds.as_numpy(nq_dataset_codeANDcomment_code("validation").take(3)):
  print(ex)
print("A few raw training examples...")
for ex in tfds.as_numpy(nq_dataset_codeANDcomment_code("train").take(3)):
  print(ex)

def codeANDcomment_code_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['code&comment2code: ' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

#Create a new training task
t5.data.TaskRegistry.remove('codeANDcomment_code')
t5.data.TaskRegistry.add(
    "codeANDcomment_code",
    dataset_fn=nq_dataset_codeANDcomment_code,
    splits=["train", "validation"],
    text_preprocessor=[codeANDcomment_code_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_codeANDcomment_code
)

nq_task = t5.data.TaskRegistry.get("codeANDcomment_code")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 512, "targets": 512})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(3)):
  print(ex)



############### FOURTH TASK ###############
print("4th TASK : marked code 2 code")

def nq_dataset_marked_code(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_marked_code[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw validation examples...")
for ex in tfds.as_numpy(nq_dataset_marked_code("validation").take(3)):
  print(ex)
print("A few raw training examples...")
for ex in tfds.as_numpy(nq_dataset_marked_code("train").take(3)):
  print(ex)

def marked_code_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['markedCode2code: ' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

#Create a new training task
t5.data.TaskRegistry.remove('marked_code')
t5.data.TaskRegistry.add(
    "marked_code",
    dataset_fn=nq_dataset_marked_code,
    splits=["train", "validation"],
    text_preprocessor=[marked_code_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_marked_code
)

nq_task = t5.data.TaskRegistry.get("marked_code")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 512, "targets": 512})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(3)):
  print(ex)

1st TASK : code 2 code
A few raw validation examples...
{'input': b'public void createRecoveryKey() { Task<String> task = new RecoveryKeyCreationTask(); task.setOnScheduled(event -> { LOG.debug("Creating recovery key for {}.", vault.getDisplayablePath()); }); task.setOnSucceeded(event -> { String recoveryKey = task.getValue(); recoveryKeyProperty.set(recoveryKey); window.setScene(successScene.get()); }); task.setOnFailed(event -> { if (task.getException() instanceof InvalidPassphraseException) { Animations.createShakeWindowAnimation(window).play(); } else { LOG.error("Creation of recovery key failed.", task.getException()); errorComponent.cause(task.getException()).window(window).returnToScene(createScene.get()).build().showErrorScene(); } }); executor.submit(task); }', 'output': b'public void createRecoveryKey() { Task<String> task = new RecoveryKeyCreationTask(); task.setOnScheduled(event -> { LOG.debug("Creating recovery key for {}.", vault.getDisplayablePath()); }); task.setOnSucce

/usr/local/lib/python3.7/dist-packages/seqio/preprocessors.py:90: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'code2code: private AdminClient brokerAdminClient(KafkaProperties kafkaProperties) { Properties props = new Properties(); props.put(BOOTSTRAP_SERVERS_CONFIG, kafkaProperties.getBootstrapKafkaServer()); props.put(SECURITY_PROTOCOL_CONFIG, DEFAULT_SECURITY_PROTOCOL); props.put(REQUEST_TIMEOUT_MS_CONFIG, kafkaProperties.getKafkaServerRequestTimeoutMillis()); if (kafkaProperties.isSaslEnabled()) { props.put("sasl.mechanism", "PLAIN"); props.put("security.protocol", "SASL_PLAINTEXT"); props.put("sasl.jaas.config", kafkaProperties.getSaslJaasConfig()); } return AdminClient.create(props); }', 'inputs': array([   89,   111,   220,    20,   124,     7, 14497,  6455, 14497,
           9, 13710,   786, 13603,   786,    12,    15,  2127,  2446,
          16,    33,  2127,    39,  2446,     8,   261,     9, 21595,
       11055,  8176,    25,  5372,   284,    25,  5379,    11, 13603,
         786,     8,    62,  8572, 13710,  1064,    

In [4]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("all_tasks")
t5.data.MixtureRegistry.add(
    "all_tasks",
    # ["code_comment"],
    ["marked_code"],
    # ["code_code"],
    # ["codeANDcomment_code"],
    # ["codeANDcomment_code", "code_code", "code_comment"],
    default_rate=_rate_num_input_examples
     #default_rate=1.0
)


from mesh_tensorflow.transformer.learning_rate_schedules import slanted_triangular 

# from mesh_tensorflow.transformer.learning_rate_schedules import truncated_rsqrt
 
# from tensorflow.keras.optimizers.schedules import PolynomialDecay

# starter_learning_rate = 0.05
# end_learning_rate = 0.001
# decay_steps = 10000

# learning_rate_fn = PolynomialDecay(
#     starter_learning_rate,
#     decay_steps,
#     end_learning_rate,
#     power=0.5)


MODEL_SIZE = "small"

############ CHANGE HERE ############
MODEL_DIR = 'gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR='gs://code_review_automation/model_dumps/'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 128, 200),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    ############ CHANGE HERE ############
    learning_rate_schedule = slanted_triangular,
    sequence_length={"inputs": 512, "targets": 512},
    save_checkpoints_steps=10000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [5]:
############ CHANGE HERE ############
!gsutil cp gs://code_review_automation/fine_tuning/slanted/operative_config.gin ./operative_config.gin 
PATH_GIN_FILE = '/content/operative_config.gin'

Copying gs://code_review_automation/fine_tuning/slanted/operative_config.gin...
/ [1 files][ 11.6 KiB/ 11.6 KiB]                                                
Operation completed over 1 objects/11.6 KiB.                                     


In [6]:
import gin

with gin.unlock_config():
    gin.parse_config_file(PATH_GIN_FILE)
    #RUN FINE-TUNING
    FINETUNE_STEPS = 300000
    model.finetune(
        mixture_or_task_name="all_tasks",
        pretrained_model_dir=PRETRAINED_DIR,
        finetune_steps=FINETUNE_STEPS
    )

INFO:root:system_path_file_exists:gs://code_review_automation/model_dumps/operative_config.gin
ERROR:root:Path not found: gs://code_review_automation/model_dumps/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.73.70.58:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.73.70.58:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.73.70.58:8470', '_evaluation_master': 

In [8]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = 1024
model.eval(
    mixture_or_task_name="all_tasks",
    checkpoint_steps=[220000, 230000, 240000, 250000, 260000, 270000, 280000, 290000, 300000, 310000, 325500, 335500, 345500, 355500, 365500, 375500, 385500, 395500, 405500, 415500, 425500, 435500, 437400, 447400, 457400, 467400, 477400, 487400, 497400, 500000]
    )

INFO:root:system_path_file_exists:gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/operative_config.gin
ERROR:root:Path not found: gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/operative_config.gin
/usr/local/lib/python3.7/dist-packages/seqio/preprocessors.py:90: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
INFO:absl:Skipping packing/padding for 'marked_code' since sequence length is None.
INFO:absl:Setting sequence lengths to {'inputs': 513, 'targets': 512}
INFO:absl:Evaluating checkpoint step: 220000


INFO:tensorflow:Using config: {'_model_dir': 'gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.73.70.58:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.73.70.58:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.73.70.58:8470', '_evaluation_master': 

INFO:absl:Padding 'marked_code' with sequence lengths: {'inputs': 513, 'targets': 512}


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('experts', 'batch'), ('batch', 'batch'), ('d_ff', 'model'), ('vocab', 'model'), ('ensemble', 'ensemble'), ('heads', 'model')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7f11ff9153d0>
INF

INFO:absl:eval/marked_code/accuracy at step 220000: 3.148
INFO:absl:Evaluating checkpoint step: 230000


...

INFO:absl:eval/marked_code/accuracy at step 497400: 6.706
INFO:absl:Evaluating checkpoint step: 500000


INFO:tensorflow:Using config: {'_model_dir': 'gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.73.70.58:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.73.70.58:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.73.70.58:8470', '_evaluation_master': 

INFO:absl:Padding 'marked_code' with sequence lengths: {'inputs': 513, 'targets': 512}


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('experts', 'batch'), ('batch', 'batch'), ('d_ff', 'model'), ('vocab', 'model'), ('ensemble', 'ensemble'), ('heads', 'model')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7f120094c890>
INF

INFO:absl:eval/marked_code/accuracy at step 500000: 6.861


In [ ]:
if ON_CLOUD:
  %reload_ext tensorboard
  import tensorboard as tb
tb.notebook.start("--logdir " + MODEL_DIR)